In [1]:
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import json

In [2]:
sims = ["unixcoder", "structural", "combined"]
metrics = ["SM", "ICP", "IFN", "NED"]
metrics_sign = {"SM": 1, "ICP": -1, "IFN": -1, "NED": -1}
cluster_sizes = [10, 18, 28]
results = {}

for method in sims:
	results[method] = {}
	with open(f"../results/Sim/jpetstore_{method}_features.json", "r") as f:
		data = json.load(f)
	df = pd.DataFrame(data).drop(["microservices"], axis=1)
	for n_clusters in cluster_sizes:
		results[method][n_clusters] = {}
		ndf = df[df["n_clusters"] == n_clusters]
		s = normalize([ndf["SM"].to_numpy()])
		for metric in metrics[1:]:
			s -= normalize([ndf[metric].to_numpy()])
		idx = np.argmax(s)
		for metric in metrics:
			results[method][n_clusters][metric] = ndf[metric].iloc[idx]

In [3]:
bests = []
for n_clusters in cluster_sizes:
	for metric in metrics:
		best_method = None
		best_metric = float('-inf') * metrics_sign[metric]
		for method in sims:
			if metrics_sign[metric] == 1:
				if results[method][n_clusters][metric] > best_metric:
					best_metric = results[method][n_clusters][metric]
					best_method = method
			else:
				if results[method][n_clusters][metric] < best_metric:
					best_metric = results[method][n_clusters][metric]
					best_method = method
		bests.append((n_clusters, metric, best_method))

In [4]:
bests

[(10, 'SM', 'combined'),
 (10, 'ICP', 'unixcoder'),
 (10, 'IFN', 'unixcoder'),
 (10, 'NED', 'unixcoder'),
 (18, 'SM', 'combined'),
 (18, 'ICP', 'unixcoder'),
 (18, 'IFN', 'structural'),
 (18, 'NED', 'unixcoder'),
 (28, 'SM', 'combined'),
 (28, 'ICP', 'structural'),
 (28, 'IFN', 'unixcoder'),
 (28, 'NED', 'unixcoder')]

In [ ]:
table_head = r'''	\begin{table}[ht]
		\centering
		\begin{tabular}{|l|ccc|ccc|ccc|ccc|}
			\hline
			\multirow{2}{*}{\textbf{features}} & \multicolumn{3}{c}{\textbf{SM $\uparrow$}} & \multicolumn{3}{|c|}{\textbf{ICP $\downarrow$}} & \multicolumn{3}{c|}{\textbf{IFN $\downarrow$}} & \multicolumn{3}{c|}{\textbf{NED $\downarrow$}} \\
%			\cline{2-13}
			& \textbf{L} & \textbf{M} & \textbf{H} & \textbf{L} & \textbf{M} & \textbf{H} & \textbf{L} & \textbf{M} & \textbf{H} & \textbf{L} & \textbf{M} & \textbf{H}\\
			\hline'''
table_tail = r'''		\hline
		\end{tabular}
		\caption{ JPetStore }
	\end{table}'''

In [6]:
print(table_head)
for method in sims:
	line = method
	for metric in metrics:
		for n_clusters in cluster_sizes:
			if (n_clusters, metric, method) in bests:
				line += f" & \\textbf{{{results[method][n_clusters][metric]:.3f}}}"
			else:
				line += f" & {results[method][n_clusters][metric]:.3f}"
	line += r"\\"
	print(line)
print(table_tail)

	\begin{table}[ht]
		\centering
		\begin{tabular}{|l|ccc|ccc|ccc|ccc|}
			\hline
			\multirow{2}{*}{\textbf{features}} & \multicolumn{3}{c}{\textbf{SM $\uparrow$}} & \multicolumn{3}{|c|}{\textbf{ICP $\downarrow$}} & \multicolumn{3}{c|}{\textbf{IFN $\uparrow$}} & \multicolumn{3}{c|}{\textbf{NED $\downarrow$}} \\
%			\cline{2-13}
			& \textbf{L} & \textbf{M} & \textbf{H} & \textbf{L} & \textbf{M} & \textbf{H} & \textbf{L} & \textbf{M} & \textbf{H} & \textbf{L} & \textbf{M} & \textbf{H}\\
			\hline
unixcoder & 0.130 & 0.102 & 0.086 & \textbf{0.078} & \textbf{0.120} & 0.173 & \textbf{0.000} & 0.056 & \textbf{0.071} & \textbf{0.547} & \textbf{0.448} & \textbf{0.442}\\
structural & 0.112 & 0.082 & 0.188 & 0.094 & 0.129 & \textbf{0.000} & 0.100 & \textbf{0.000} & 0.333 & 0.965 & 0.847 & 1.000\\
combined & \textbf{0.146} & \textbf{0.136} & \textbf{0.198} & 0.089 & 0.137 & 0.214 & 0.100 & 0.056 & 0.071 & 0.828 & 0.641 & 0.524\\
		\hline
		\end{tabular}
		\caption{ JPetStore }
	\end{table}
